In [1]:
using LinearAlgebra, Symbolics

In [2]:
@variables r s;

In [3]:
ν = 0.3 # Coeficiente de Poisson
EM = 2.1E06 # Módulo de Young - kgf/cm^2
C = (EM/(1-ν^2)) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]; # Matriz de constantes elásticas (Tensión plana)

In [4]:
NodalCoords = [0 0; 3 0; 1.5 3] # Coordenadas Globales para elemento de 3 nodos

3×2 Matrix{Float64}:
 0.0  0.0
 3.0  0.0
 1.5  3.0

In [5]:
# Funciones de interpolación para un elemento isoparamétrico triangular de 3 nodos
h1 = 1-r-s
h2 = r
h3 = s;

In [6]:
Dr = Differential(r)
Ds = Differential(s)

(::Differential) (generic function with 2 methods)

In [7]:
H = [h1 h2 h3]
H1 = [h1 h2 h3; h1 h2 h3]
HM = [h1 0 h2 0 h3 0 ; 0 h1 0 h2 0 h3]
display(H)

1×3 Matrix{Num}:
 1 - r - s  r  s

In [8]:
∇H = [transpose([expand_derivatives.(Dr(H1[1,i])) for i in 1:3]); transpose([expand_derivatives.(Ds(H1[2,i])) for i in 1:3])]

2×3 Matrix{Num}:
 -1  1  0
 -1  0  1

In [9]:
x = expand((H * NodalCoords[:,1])[1])
y = expand((H * NodalCoords[:,2])[1])
display(x)
display(y)

1.5s + 3.0r

3.0s

In [10]:
dxdr = expand_derivatives(Dr(x))
dxds = expand_derivatives(Ds(x))        
dydr = expand_derivatives(Dr(y))
dyds = expand_derivatives(Ds(y))

3.0

In [11]:
J = [dxdr dydr; dxds dyds] # Jacobiano
J_inv = inv(J) # Inversa del Jacobiano
J_det = det(J) # Determinante del Jacobiano
display(J)
display(J_inv)
display(J_det)

2×2 Matrix{Num}:
 3.0  0
 1.5  3.0

2×2 Matrix{Num}:
  0.333333  0.0
 -0.166667  0.333333

9.0

In [12]:
B1 = transpose(∇H) * J_inv

3×2 Matrix{Num}:
 -0.166667  -0.333333
  0.333333   0.0
 -0.166667   0.333333

In [13]:
B = [B1[1,1] 0 B1[2,1] 0 B1[3,1] 0 
    0 B1[1,2] 0 B1[2,2] 0 B1[3,2] 
    B1[1,2] B1[1,1] B1[2,2] B1[2,1] B1[3,2] B1[3,1]]

3×6 Matrix{Num}:
 -0.166667   0         0.333333  0         -0.166667   0
  0         -0.333333  0         0.0        0          0.333333
 -0.333333  -0.166667  0.0       0.333333   0.333333  -0.166667

In [14]:
GL3T = [3/18 3/18; 2/3 3/18; 3/18 2/3] # Puntos de Gauss para integración

3×2 Matrix{Float64}:
 0.166667  0.166667
 0.666667  0.166667
 0.166667  0.666667

In [15]:
WGL3T = [1/3 1/3; 1/3 1/3; 1/3 1/3]

3×2 Matrix{Float64}:
 0.333333  0.333333
 0.333333  0.333333
 0.333333  0.333333

In [24]:
K = zeros(6,6) 

#Integramos por medio de cuadratura de Gauss para tres puntos sobre dominios triangulares
for i in 1:3        
        global Jg = substitute(J, Dict([r => GL3T[i,1], s => GL3T[i,2]]))        
        Jg_inv = substitute(J_inv, Dict([r => GL3T[i,1], s => GL3T[i,2]]))
        Jg_det = substitute(J_det, Dict([r => GL3T[i,1], s => GL3T[i,2]]))                
        Bg = substitute(B, Dict([r => GL3T[i,1], s => GL3T[i,2]]))                    
        K += (transpose(Bg) * C * Bg * Jg_det * WGL3T[i,1] * WGL3T[i,2])
        
    end
    display(Jg)
display(K)

2×2 Matrix{Num}:
 3.0  0
 1.5  3.0

6×6 Matrix{Num}:
      4.61538e5  250000.0        …   -76923.1          19230.8
 250000.0             8.36538e5      -19230.8             -7.01923e5
     -3.84615e5      -2.30769e5          -3.84615e5        2.30769e5
     -2.69231e5      -1.34615e5           2.69231e5       -1.34615e5
 -76923.1        -19230.8                 4.61538e5  -250000.0
  19230.8            -7.01923e5  …  -250000.0              8.36538e5

In [17]:
 # Imponemos condiciones de contorno de Dirichlet
 # U1x=0 => U1=0
    K[1,:] .= 0
    K[:,1] .= 0
    K[1,1] =1
 # U2x=0 => U3=0
    K[3,:] .= 0
    K[:,3] .= 0
    K[3,3] =1
 # U2y=0 => U4=0
    K[4,:] .= 0
    K[:,4] .= 0
    K[4,4] =1

display(K)

6×6 Matrix{Num}:
 1       0          0  0        0                0
 0       8.36538e5  0  0   -19230.8             -7.01923e5
 0       0          1  0        0                0
 0       0          0  1        0                0
 0  -19230.8        0  0        4.61538e5  -250000.0
 0      -7.01923e5  0  0  -250000.0              8.36538e5

In [18]:
#= \(A,B)
Matrix division using a polyalgorithm. For input matrices A and B, the result X is such that A*X == B when A is square. 
The solver that is used depends upon the structure of A. If A is upper or lower triangular (or diagonal), no factorization 
of A is required and the system is solved with either forward or backward substitution. For non-triangular square matrices, 
an LU factorization is used.
=#

In [19]:
# CASO 1. F1 = 500kgf ; F2 = 0kgf

R1 =[0, 0, 0, 0, 500, 0] # Fuerzas nodales en kgf.
U1 = K \ R1 # U1 --> Desplazamientos nodales.
E1 = B * U1
E1[3] = E1[3]/2
σ1 = C * E1
display(E1)
display(σ1)

3-element Vector{Num}:
 -0.0004761904761904749
  0.0001428571428571425
  7.589415207398531e-19

3-element Vector{Num}:
 -999.9999999999972
    5.684341886080802e-14
    6.129912282898813e-13

In [20]:
# CASO 2. F1 = 0kgf ; F2 = 500kgf

R2 =[0, 0, 0, 0, 0, 500] # Fuerzas nodales en kgf.
U2 = K \ R2 # U1 --> Desplazamientos nodales.
E2 = B * U2
E2[3] = E2[3]/2
σ2 = C * E2

3-element Vector{Num}:
 -999.9999999999957
  249.99999999999966
 -249.9999999999991

In [21]:
# CASO 3. F1 = 500kgf ; F2 = 500kgf

R3 =[0, 0, 0, 0, 500, 500] # Fuerzas nodales en kgf.
U3 = K \ R3 # U1 --> Desplazamientos nodales.
E3 = B * U3
E3[3] = E3[3]/2
σ3 = C * E3

3-element Vector{Num}:
 -1999.999999999993
   250.00000000000023
  -249.99999999999844